In [2]:
import pandas as pd
import numpy as np
import requests
import time
import re
import os
from request_simec import login, get_url, post_url
from restrictions import exemplares_aceitos, formatos_aceitos

import zipfile
from PyPDF2 import PdfFileReader
from pathlib import Path

import hashlib
import math
import shutil
import unidecode
import sys

# Download dos Arquivos

In [41]:
#arquivo inicial extraído diretamente do site do simec
df = pd.read_csv(
         'dados/TODOS.csv',
         dtype=str
    )

In [24]:
def login_simec ():
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    s = requests.Session()
    s.get(get_url)
    s.post(post_url, data=login)
    
    return s

In [ ]:
s = login_simec()

In [ ]:
def verify_reds_simec():
    for index, row in df.iterrows():
    #condição para o arquivo ser baixado
    if (row['Formato'] in formatos_aceitos) and (row['Exemplar'] in exemplares_aceitos):
        df.at[index, 'Deve ser baixado'] = 'Sim'
        df.at[index, 'Baixado'] = 'Não'

        #verificar head do arquivo no servidor
        header = s.head(row['Arquivo'])
        df.at[index, 'Header'] = header.headers
    
        #mapeando nome e extensão do arquivo via header
        if "content-disposition" in header.headers.keys():
            content_disposition = header.headers['content-disposition']
            try:
                f_name = re.findall("filename=(.+)", content_disposition)[0]
            except IndexError:
                f_name = content_disposition      
        else:
            f_name = "NO CONTENT-DISPOSITION"
            
        if "Content-Type" in header.headers.keys():
            content_type = header.headers['Content-Type']
            try:
                c_type = re.findall("application/(.+)", content_type)[0]
            except IndexError:
                c_type = content_type
        else:
            c_type = "NO CONTENT-TYPE"
            
        df.at[index, 'Nome do Arquivo'] = f_name
        df.at[index, 'Extensão do Arquivo'] = c_type
        
        print(row['id'] + ' ' + f_name + ' ' + c_type)

    else:
        df.at[index, 'Deve ser baixado'] = 'Não'
        print(row['id'] + ' ****** ' + 'NÃO DEVE SER BAIXADO')
        

In [ ]:
def download_reds() {
    i = 1
    for index, row in df.iterrows():  
        if (row['Deve ser baixado'] == 'Sim') and (row['Baixado'] == 'Não'):
            print(str(i) + ' - ' + row['id'] + ' ' + row['Nome do Arquivo'] + ' ' + row['Extensão do Arquivo'])

            #Baixa o arquivo
            try:
                r = s.get(row['Arquivo'], headers=headers)
            except Exception as e:
                print(e)
                continue

            print(r.headers)

            #Propriedade que indica erro no arquivo baixado
            if "Content-Lenght" in r.headers.keys():
                print('Badly download')
                continue

            #Salva o arquivo localmente
            filename = 'arquivos/' + row['Edital'] + '/' + row['id'] + '/' + row['Nome do Arquivo']
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, "wb") as f:
                f.write(r.content)

            #Salva nos dados o estado de Baixado e de Verificar integridade
            df.at[index, 'Baixado'] = 'Sim'
            df.at[index, 'Integridade'] = 'Verificar'
            df.to_csv('dados/TODOS.csv', index=False)
            
            print('BAIXADO!')            

            #Evita que o servidor do MEC barre o download
            time.sleep(2)
            s.get(get_url)    
            time.sleep(5)

            i += 1
}

In [ ]:
verify_reds_simec()

In [ ]:
download_reds()

# Extração e mapeamento dos zips 

In [ ]:
df = pd.read_csv(
    'dados/TODOS.csv',
    dtype=str
)

arq_zip = pd.DataFrame()

## Mapeamento do tamanho de cada arquivo e verificação se o arquivo é integro

In [ ]:
def get_all_file_sizes (df):
    for index, row in df.iterrows():
        if row['Baixado'] == 'Sim':
            path = 'arquivos/' + row['Edital'] + '/' + row['id']
            print(path);
            get_size_file_in_kb(path, df, index)


def get_size_file_in_kb(path, df, index):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.zip'):
                
                get_real_and_zipped_size_in_kb(path, df, index, file)
            else:
                get_size_file_in_kb(path, df, index, file)

def get_size_file_in_kb(path, df, index, file):
    for root, dirs, files in os.walk(path):
        if file.endswith('.pdf'):
            valid_file = check_pdf_file(path + '/' + file)
            if (valid_file):
                df.at[index, 'Integridade'] = 'ok'
                print (df.at[index, 'id'] + " - OK")
            else:
                df.at[index, 'Integridade'] = 'no a pdf file'
                print (df.at[index, 'id'] + " - BAD PDF")
                df.to_csv('dados/TODOS.csv', index = False)
                continue
                        
        file_obj = Path(path + '/' + file)
        size = file_obj.stat().st_size

        size_kb = float(size) / 1000  # kB
        print (df.at[index, 'id'] + " - " + str(size_kb))
        df.at[index, 'Size'] = size_kb
        df.to_csv('dados/TODOS.csv', index = False)

        print('\n\n')


def get_real_and_zipped_size_in_kb(path, df, index, file):
    try:
        zfile = zipfile.ZipFile(path + '/' + file)
        zipped_size = os.path.getsize(path + '/' + file)
    except zipfile.BadZipfile as ex:
        print (df.at[index, 'id'] + " no a zip file: " + ex)
        df.at[index, 'Integridade'] = 'no a zip file'
        continue
    except:

    #zip test
    ret = zfile.testzip()
    if ret is not None:
        print (df.at[index, 'id'] + " bad zip file, error: " + ret)
        df.at[index, 'Integridade'] = 'bad zip file'
    else:
        df.at[index, 'Integridade'] = 'ok'

    real_size = sum([zinfo.file_size for zinfo in zfile.filelist])
    real_size_kb = float(size) / 1000  # kB
    zipped_size_kb = float(zipped_size) / 1000  # kB

    print (df.at[index, 'id'] + " - " + str(zip_kb))
    df.at[index, 'Size'] = real_size_kb
    df.at[index, 'Zipped size'] = zipped_size_kb
    df.to_csv('dados/TODOS.csv', index = False)


def check_pdf_file(fullfile):
    with open(fullfile, 'rb') as f:
        try:
            pdf = PdfFileReader(f)
            info = pdf.getDocumentInfo()
            if info:
                return True
            else:
                return False
        except:
            return False

In [ ]:
get_all_file_sizes (df)

## Descarte de arquivos não integros/corrompidos

In [ ]:
def remove_corrupted_files():
    i = 0
    for index, row in df.iterrows():
        if (row['Integridade'] != 'ok'):
            file = 'arquivos/' + row['Edital'] + '/' + row['id'] + '/' + row['Nome do Arquivo']
            print(row['Edital'] + '/' + row['id'])
            print(row['Arquivo'])
            i = i + 1
            if os.path.exists(file):
                os.remove(file)
                print("Arquivo Removido")
            else:
                print("The file does not exist")

            df.at[index, 'Baixado'] = 'Não'
            df.to_csv('dados/TODOS.csv', index = False)
            print('\n\n')
    
    print(i)

In [ ]:
remove_corrupted_files()

## Mapeamento dos Arquivos zip e seus metadados próprios

In [ ]:
def map_files_in_zips (df, arq_zip):
    i = 0
    for index, row in df.iterrows():
        if (row['Baixado'] == 'Sim') and (row['Extensão do Arquivo'] in ['zip', 'x-zip-compressed']):
            path = 'arquivos/' + row['Edital'] + '/' + row['id']
#             print(path);
            
            i = get_all_files_in_zip(path, df, index, row['id'], arq_zip, i)


def get_all_files_in_zip(path, df, index, df_row_id, arq_zip, i):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.zip'):
                with zipfile.ZipFile(path + '/' + file, 'r') as zipObj:
                    listOfiles = zipObj.infolist()
#                     print(listOfiles)
                    for elem in listOfiles:
                        row = {}
                        row['red_id'] = df_row_id
                        row['nome'] = elem.filename.split("/")[-1]
                        row['tamanho'] = elem.file_size
                        row['tamanho_zipado'] = elem.compress_size
                        row['extensão'] = elem.filename.split(".")[-1]
                        row['path'] = elem.filename
                        print(str(i) + ' - ', end="")
                        print(row)
                        arq_zip.loc[i] = row
                        arq_zip.to_csv('dados/arquivos_zip.csv', index = False)
        
                        i = i + 1
            else:
                print('ERROR: ISNT ZIP')
                
    return i

In [ ]:
map_files_in_zips (df, arq_zip)

## Unzip dos arquivos

In [ ]:
def unzip_all (df):
    for index, row in df.iterrows():
        if(row['Extensão do Arquivo'] in ['zip', 'x-zip-compressed']):
#             print (row['id'] + ' - ' + row['Edital'])
#             row['Nome do Arquivo']
            path = 'arquivos/' + row['Edital'] + '/zip/' + row['id']
#             path = row['id']
            
            with zipfile.ZipFile(path + '/' + row['Nome do Arquivo'], "r") as zip_ref:
                zip_ref.extractall(path)
            os.remove(path + '/' + row['Nome do Arquivo'])
            
            df.at[index, 'Unzip'] = 'ok'



In [ ]:
unzip_all(df)

## Padronização dos dados dos arquivos zip para os dados dos outros arquivos

In [ ]:
ren = arq_zip.rename(columns={'red_id': 'id', 'nome': 'Nome do Arquivo', 'tamanho': 'Size',
                       'tamanho zipado': 'Zipped size', 'extensão': 'Extensão do Arquivo',
                       'path': 'Caminho Local'})
ren = ren[ren['Extensão do Arquivo'] != 'DS_Store']


### Inserção dos metadados do zip nos arquivos descompactados do mesmo

In [ ]:

for index, row in df.iterrows():
    if (row['Extensão do Arquivo'] in ['zip', 'x-zip-compressed']):
        #print('rowz')
        rows_zip = ren[ren['id'] == row['id']]
        #print(rows_zip)
        i = 1
        for i_zip, r_zip in rows_zip.iterrows():
            alter_dataframe_register(i_zip, row, r_zip, i)
            i += 1

def alter_dataframe_register(index, origin_row, destiny_row, i):
    ren.at[index, 'Edital'] = origin_row['Edital']
    ren.at[index, 'CNPJ Editora'] = origin_row['CNPJ Editora']
    ren.at[index, 'Editora'] = origin_row['Editora']
    ren.at[index, 'Código Coleção'] = origin_row['Código Coleção']
    ren.at[index, 'Coleção'] = origin_row['Coleção']
    ren.at[index, 'Código Volume'] = origin_row['Código Volume']
    ren.at[index, 'Volume'] = origin_row['Volume']
    ren.at[index, 'Componente'] = origin_row['Componente']
    ren.at[index, 'Série'] = origin_row['Série']
    ren.at[index, 'Formato'] = origin_row['Formato']
    ren.at[index, 'Exemplar'] = origin_row['Exemplar']
    ren.at[index, 'Tipo de Arquivo'] = origin_row['Tipo de Arquivo']
    ren.at[index, 'Arquivo'] = origin_row['Arquivo']
    ren.at[index, 'Header'] = origin_row['Header']
    #ren.at[index, 'Header get()'] = origin_row['Header get()']
    ren.at[index, 'Deve ser baixado'] = origin_row['Deve ser baixado']
    ren.at[index, 'Baixado'] = origin_row['Baixado']
    #ren.at[index, 'Extensão do Arquivo no Header'] = origin_row['Extensão do Arquivo no Header']
    ren.at[index, 'Integridade'] = origin_row['Integridade']
    ren.at[index, 'id_zip'] = i
    print(ren.at[index, 'Edital'])


def set_caminho_local():
    for index, row in df.iterrows():
        if (row['Formato'] in ['Digital', 'Audiovisual']):
            if (type(row['id_zip']) is str):
                #print('rowz')
                rows_zip = ren[ren['id'] == row['id']]
                #print(rows_zip)
                for i_zip, r_zip in rows_zip.iterrows():
                    if (r_zip['Nome do Arquivo'] != row['Nome do Arquivo']):
                        continue

                    print(r_zip['Nome do Arquivo'] + ' - ' + row['Nome do Arquivo'])
                    df.at[index, 'Caminho Local'] = r_zip['Caminho Local']
                    print (df.at[index, 'Caminho Local'])
                    print('OK')

In [ ]:
set_caminho_local()

## Cruzamento dos dados dos arquivos descompactados com os outros arquivos baixados

In [ ]:
m = pd.concat([df, ren])
m = m[(m['Extensão do Arquivo'] != 'zip') & (m['Extensão do Arquivo'] != 'x-zip-compressed')]

ren.to_csv('dados/arquivos_zip.csv', index = False)
m.to_csv('dados/TODOS.csv', index = False)

In [ ]:
df = pd.read_csv(
    'dados/TODOS.csv',
    dtype=str
)

In [ ]:
df.loc[df['Edital'] == 'PNLD 2020 - Obras Didáticas - Anos Finais do Ensino Fundamental', ['edital sv']] =  'pnld_2020'
df.loc[df['Edital'] == 'PNLD 2019 - ATUALIZAÇÃO BNCC - Educação Infantil e Anos Iniciais do Ensino Fundamental', ['edital sv']] =  'pnld_2019'